# 📘 Day 1: ML Model Deployment

**🎯 Goal:** Master deploying machine learning models to production

**⏱️ Time:** 120-150 minutes

**🌟 Why This Matters for AI (2024-2025):**
- A model is USELESS if it can't be deployed - deployment is where value is created
- 87% of ML projects never make it to production - learn deployment to be in the 13%!
- Flask and FastAPI are THE industry standards for ML APIs
- Docker is essential for reproducible, scalable deployments
- Every AI company deploys models as REST APIs for real-time predictions
- Deployment skills separate hobbyists from professional ML engineers

**What You'll Build Today:**
1. **Save and load ML models** using joblib and pickle
2. **Build a Flask API** for ML model serving
3. **Create a FastAPI service** for production-grade deployment
4. **Containerize with Docker** for reproducibility
5. **Deploy a real sentiment analysis API** from scratch

---

## 🌍 ML Deployment Landscape (2024-2025)

**From Jupyter Notebook to Production!**

### 🎯 Deployment Strategies:

#### 🔧 **1. Real-Time (Online) Serving**

**What:** Instant predictions on-demand via API

**How:**
- REST API (Flask, FastAPI)
- gRPC for high performance
- WebSocket for streaming

**Use Cases:**
- Chatbots (immediate responses)
- Recommendation systems (real-time suggestions)
- Fraud detection (instant decision)
- Image classification (upload and classify)

**Pros:** Immediate results, interactive  
**Cons:** Higher latency, more expensive

#### 📦 **2. Batch Prediction**

**What:** Process large datasets periodically

**How:**
- Scheduled jobs (Airflow, cron)
- Spark for big data
- Cloud batch services

**Use Cases:**
- Email campaigns (score all users nightly)
- Risk assessment (monthly credit scoring)
- Report generation (weekly forecasts)

**Pros:** Efficient, cheaper  
**Cons:** Not real-time

#### ⚡ **3. Edge Deployment**

**What:** Run models on devices (phones, IoT)

**How:**
- TensorFlow Lite
- ONNX Runtime
- CoreML (iOS)

**Use Cases:**
- Mobile apps (face recognition)
- IoT sensors (anomaly detection)
- Autonomous vehicles

**Pros:** Ultra-fast, privacy  
**Cons:** Limited compute

#### 🌐 **4. Serverless**

**What:** Pay-per-request, auto-scaling

**How:**
- AWS Lambda
- Google Cloud Functions
- Azure Functions

**Use Cases:**
- Sporadic requests
- Microservices
- Event-driven ML

**Pros:** No servers, auto-scale  
**Cons:** Cold starts, limits

### 📊 Choosing a Strategy:

| Need | Strategy | Why |
|------|----------|-----|
| **Instant results** | Real-time API | User waiting |
| **Million predictions** | Batch | Efficient |
| **Low latency** | Edge | On-device |
| **Variable traffic** | Serverless | Auto-scale |
| **Chat/interactive** | Real-time | Conversational |
| **Cost-sensitive** | Batch | Cheaper |

**Today's focus: Real-time APIs** (most common deployment pattern)

Let's build!

---

## 🛠️ Setup & Installation

**Install required libraries:**

In [ ]:
# Install required libraries
import sys

# Core ML libraries
!{sys.executable} -m pip install scikit-learn numpy pandas joblib --quiet

# Web frameworks
!{sys.executable} -m pip install flask fastapi uvicorn[standard] requests --quiet

# For text processing
!{sys.executable} -m pip install nltk transformers torch --quiet

# Visualization
!{sys.executable} -m pip install matplotlib seaborn --quiet

print("✅ Libraries installed successfully!")
print("\n💡 Docker installation: https://docs.docker.com/get-docker/")
print("   (Docker is optional for this notebook but recommended for production)")

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Set random seed
np.random.seed(42)

print("📦 Libraries imported successfully!")
print("🚀 Ready to deploy ML models!\n")

## 💾 Step 1: Model Serialization (Saving & Loading)

**Before deployment, you need to SAVE your trained model!**

### 🎯 Why Save Models?

- ✅ **No re-training**: Train once, deploy many times
- ✅ **Version control**: Save different model versions
- ✅ **Reproducibility**: Exact same predictions
- ✅ **Sharing**: Share models with team/production

### 📊 Saving Methods:

| Method | Library | Use Case |
|--------|---------|----------|
| **Joblib** | scikit-learn | Sklearn models (recommended) |
| **Pickle** | Python built-in | Any Python object |
| **SavedModel** | TensorFlow | TF/Keras models |
| **torch.save** | PyTorch | PyTorch models |
| **ONNX** | Cross-platform | Framework-agnostic |

**Best practice: Use joblib for sklearn (faster for large numpy arrays)**

In [ ]:
# Create and train a simple sentiment analysis model

print("🎯 Training Sentiment Analysis Model\n")
print("="*70)

# Sample dataset (in production, use real datasets like IMDB)
texts = [
    "I love this product! It's amazing!",
    "Terrible experience, very disappointed.",
    "Great quality and fast shipping!",
    "Worst purchase ever, don't buy.",
    "Absolutely fantastic, highly recommend!",
    "Poor quality, broke after one day.",
    "Exceeded my expectations, wonderful!",
    "Waste of money, completely useless.",
    "Best decision ever, love it!",
    "Horrible customer service and product.",
    "Outstanding quality, will buy again!",
    "Not worth the price, very bad.",
    "Perfect! Exactly what I needed.",
    "Disappointing and overpriced.",
    "Amazing product, works perfectly!",
    "Awful quality, returned immediately.",
    "Superb! Better than expected.",
    "Garbage product, total scam.",
    "Incredible value, very satisfied!",
    "Terrible quality, broke quickly."
]

labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1=positive, 0=negative

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

print(f"📊 Dataset:")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f"\n✅ Vectorizer created:")
print(f"   Vocabulary size: {len(vectorizer.vocabulary_)}")

# Train logistic regression model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_vec, y_train)

# Evaluate
y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n✅ Model trained!")
print(f"   Accuracy: {accuracy:.2%}")

# Test predictions
print(f"\n🧪 Test Predictions:\n")
test_samples = [
    "This is awesome!",
    "I hate this product",
    "Pretty good, I like it"
]

for text in test_samples:
    vec = vectorizer.transform([text])
    pred = model.predict(vec)[0]
    prob = model.predict_proba(vec)[0]
    sentiment = "Positive 😊" if pred == 1 else "Negative 😞"
    print(f"{sentiment} ({prob[pred]:.2%} confidence)")
    print(f"   Text: \"{text}\"")
    print()

print("="*70)
print("\n💡 Now let's SAVE this model for deployment!")

In [ ]:
# Save model and vectorizer using joblib

print("💾 Saving Model & Vectorizer\n")
print("="*70)

# Create models directory
import os
os.makedirs('models', exist_ok=True)

# Save with joblib (recommended for sklearn)
joblib.dump(model, 'models/sentiment_model.joblib')
joblib.dump(vectorizer, 'models/vectorizer.joblib')

print("✅ Saved with joblib:")
print("   📄 models/sentiment_model.joblib")
print("   📄 models/vectorizer.joblib")

# Save with pickle (alternative method)
with open('models/sentiment_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('models/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print("\n✅ Also saved with pickle:")
print("   📄 models/sentiment_model.pkl")
print("   📄 models/vectorizer.pkl")

# Save metadata
metadata = {
    'model_type': 'LogisticRegression',
    'accuracy': float(accuracy),
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'num_features': len(vectorizer.vocabulary_),
    'classes': ['negative', 'positive']
}

with open('models/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("\n✅ Metadata saved:")
print("   📄 models/metadata.json")

# Check file sizes
model_size = os.path.getsize('models/sentiment_model.joblib') / 1024
vec_size = os.path.getsize('models/vectorizer.joblib') / 1024

print(f"\n📊 File Sizes:")
print(f"   Model: {model_size:.2f} KB")
print(f"   Vectorizer: {vec_size:.2f} KB")

print("\n" + "="*70)
print("\n💡 Models saved! Now they can be loaded in production without re-training.")

In [ ]:
# Load model and test (simulating production environment)

print("📂 Loading Saved Model\n")
print("="*70)

# Load model and vectorizer
loaded_model = joblib.load('models/sentiment_model.joblib')
loaded_vectorizer = joblib.load('models/vectorizer.joblib')

print("✅ Model and vectorizer loaded!")

# Load metadata
with open('models/metadata.json', 'r') as f:
    loaded_metadata = json.load(f)

print("\n📋 Model Metadata:")
for key, value in loaded_metadata.items():
    print(f"   {key}: {value}")

# Test loaded model
print("\n🧪 Testing Loaded Model:\n")

test_texts = [
    "This is the best thing ever!",
    "Absolutely terrible, very bad",
    "It's okay, nothing special"
]

for text in test_texts:
    vec = loaded_vectorizer.transform([text])
    pred = loaded_model.predict(vec)[0]
    prob = loaded_model.predict_proba(vec)[0]
    
    sentiment = loaded_metadata['classes'][pred]
    emoji = "😊" if pred == 1 else "😞"
    
    print(f"{emoji} {sentiment.upper()} ({prob[pred]:.2%})")
    print(f"   Text: \"{text}\"")
    print()

print("="*70)
print("\n✅ Loaded model works perfectly!")
print("\n💡 This is EXACTLY how models are loaded in production APIs!")

## 🌐 Step 2: Flask API for ML Models

**Flask = Simple, lightweight web framework for Python**

### 🎯 Why Flask?

✅ **Easy to learn**: Minimal boilerplate  
✅ **Lightweight**: Perfect for simple APIs  
✅ **Widely used**: Huge community  
✅ **Flexible**: Add only what you need  

### 🏗️ Flask ML API Architecture:

```
Client (Browser/App)
       ↓
   POST /predict
   {"text": "I love this!"}
       ↓
   Flask API
       ↓
   1. Load model
   2. Preprocess input
   3. Make prediction
   4. Return JSON response
       ↓
   {"sentiment": "positive", "confidence": 0.95}
```

### 📝 Key Components:

**1. Routes:** Define API endpoints  
**2. Request handling:** Parse incoming data  
**3. Model inference:** Load model, predict  
**4. Response:** Return JSON results  

Let's build a Flask API!

In [ ]:
# Create Flask app for sentiment analysis
# Save this as 'app.py' to run: python app.py

flask_app_code = '''
from flask import Flask, request, jsonify
import joblib
import json

# Initialize Flask app
app = Flask(__name__)

# Load model and vectorizer at startup (once)
print("Loading model...")
model = joblib.load('models/sentiment_model.joblib')
vectorizer = joblib.load('models/vectorizer.joblib')
with open('models/metadata.json', 'r') as f:
    metadata = json.load(f)
print("Model loaded successfully!")

# Health check endpoint
@app.route('/', methods=['GET'])
def home():
    """Health check endpoint"""
    return jsonify({
        'status': 'online',
        'model': metadata['model_type'],
        'version': '1.0',
        'message': 'Sentiment Analysis API is running!'
    })

# Prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    """Predict sentiment from text"""
    try:
        # Get input data
        data = request.get_json()
        
        # Validate input
        if 'text' not in data:
            return jsonify({'error': 'Missing "text" field'}), 400
        
        text = data['text']
        
        if not text or not isinstance(text, str):
            return jsonify({'error': 'Invalid text input'}), 400
        
        # Preprocess and predict
        text_vec = vectorizer.transform([text])
        prediction = model.predict(text_vec)[0]
        probabilities = model.predict_proba(text_vec)[0]
        
        # Format response
        sentiment = metadata['classes'][prediction]
        confidence = float(probabilities[prediction])
        
        return jsonify({
            'text': text,
            'sentiment': sentiment,
            'confidence': confidence,
            'probabilities': {
                'negative': float(probabilities[0]),
                'positive': float(probabilities[1])
            }
        })
    
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Batch prediction endpoint
@app.route('/predict_batch', methods=['POST'])
def predict_batch():
    """Predict sentiment for multiple texts"""
    try:
        data = request.get_json()
        
        if 'texts' not in data:
            return jsonify({'error': 'Missing "texts" field'}), 400
        
        texts = data['texts']
        
        if not isinstance(texts, list):
            return jsonify({'error': '"texts" must be a list'}), 400
        
        # Process all texts
        results = []
        for text in texts:
            text_vec = vectorizer.transform([text])
            prediction = model.predict(text_vec)[0]
            probabilities = model.predict_proba(text_vec)[0]
            
            results.append({
                'text': text,
                'sentiment': metadata['classes'][prediction],
                'confidence': float(probabilities[prediction])
            })
        
        return jsonify({'predictions': results})
    
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Run app
    app.run(host='0.0.0.0', port=5000, debug=True)
'''

# Save Flask app to file
with open('app.py', 'w') as f:
    f.write(flask_app_code)

print("📝 Flask App Created!\n")
print("="*70)
print("\n✅ Saved to: app.py")
print("\n🚀 To run the Flask API:")
print("   1. python app.py")
print("   2. API will run on http://localhost:5000")
print("\n📡 API Endpoints:")
print("   GET  /           - Health check")
print("   POST /predict    - Single prediction")
print("   POST /predict_batch - Batch predictions")
print("\n💡 Example request:")
print('''   curl -X POST http://localhost:5000/predict \\''')
print('''        -H "Content-Type: application/json" \\''')
print('''        -d '{"text": "I love this product!"}\'\n''')
print("="*70)

In [ ]:
# Test Flask API (using requests library)
# Note: This assumes Flask app is running on localhost:5000

import requests

print("🧪 Testing Flask API\n")
print("="*70)
print("\n⚠️  Make sure Flask app is running: python app.py")
print("\nIf the app is running, uncomment the code below to test:\n")

test_code = '''
# Test health check
response = requests.get('http://localhost:5000/')
print("✅ Health Check:")
print(json.dumps(response.json(), indent=2))

# Test single prediction
response = requests.post(
    'http://localhost:5000/predict',
    json={'text': 'This product is amazing!'}
)
print("\n✅ Single Prediction:")
print(json.dumps(response.json(), indent=2))

# Test batch prediction
response = requests.post(
    'http://localhost:5000/predict_batch',
    json={'texts': [
        'I love this!',
        'This is terrible',
        'Pretty good product'
    ]}
)
print("\n✅ Batch Prediction:")
print(json.dumps(response.json(), indent=2))
'''

print("# " + "\n# ".join(test_code.split("\n")))

print("\n" + "="*70)
print("\n💡 Flask is great for prototypes, but FastAPI is better for production!")

## ⚡ Step 3: FastAPI for Production

**FastAPI = Modern, high-performance web framework**

### 🎯 Why FastAPI? (The 2024-2025 Standard)

✅ **FAST**: 3-4x faster than Flask  
✅ **Auto docs**: Swagger UI built-in  
✅ **Type hints**: Automatic validation  
✅ **Async**: Handle concurrent requests  
✅ **Modern**: Based on Python 3.7+ features  

### 📊 Flask vs FastAPI:

| Feature | Flask | FastAPI |
|---------|-------|----------|
| **Speed** | Moderate | Very Fast |
| **Documentation** | Manual | Auto-generated |
| **Validation** | Manual | Automatic |
| **Async** | Limited | Full support |
| **Learning Curve** | Easy | Moderate |
| **Best For** | Simple APIs | Production APIs |

**In 2024-2025: FastAPI is the industry standard for ML APIs!**

### 🌟 FastAPI Features:

**1. Automatic Documentation**
- Swagger UI at `/docs`
- ReDoc at `/redoc`
- Interactive testing

**2. Type Validation**
- Pydantic models
- Automatic type checking
- Better error messages

**3. Performance**
- Async/await support
- Fast JSON serialization
- Production-ready

Let's build a FastAPI service!

In [ ]:
# Create FastAPI app for sentiment analysis
# Save this as 'main.py' to run: uvicorn main:app --reload

fastapi_app_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from typing import List
import joblib
import json
from datetime import datetime

# Initialize FastAPI app
app = FastAPI(
    title="Sentiment Analysis API",
    description="Real-time sentiment analysis using ML",
    version="1.0.0"
)

# Load model at startup
print("🚀 Loading ML model...")
model = joblib.load('models/sentiment_model.joblib')
vectorizer = joblib.load('models/vectorizer.joblib')
with open('models/metadata.json', 'r') as f:
    metadata = json.load(f)
print("✅ Model loaded successfully!")

# Pydantic models for request/response validation
class PredictionRequest(BaseModel):
    text: str = Field(..., min_length=1, description="Text to analyze")
    
    class Config:
        schema_extra = {
            "example": {
                "text": "I love this product!"
            }
        }

class PredictionResponse(BaseModel):
    text: str
    sentiment: str
    confidence: float
    probabilities: dict
    timestamp: str

class BatchRequest(BaseModel):
    texts: List[str] = Field(..., min_items=1, description="List of texts")
    
    class Config:
        schema_extra = {
            "example": {
                "texts": ["Great product!", "Terrible quality"]
            }
        }

class BatchResponse(BaseModel):
    predictions: List[PredictionResponse]
    count: int

# Health check endpoint
@app.get("/")
async def root():
    """Health check and API information"""
    return {
        "status": "online",
        "model": metadata['model_type'],
        "accuracy": metadata['accuracy'],
        "version": "1.0.0",
        "endpoints": {
            "docs": "/docs",
            "predict": "/predict",
            "batch": "/predict_batch"
        }
    }

# Single prediction endpoint
@app.post("/predict", response_model=PredictionResponse)
async def predict(request: PredictionRequest):
    """Predict sentiment for a single text"""
    try:
        # Vectorize input
        text_vec = vectorizer.transform([request.text])
        
        # Make prediction
        prediction = model.predict(text_vec)[0]
        probabilities = model.predict_proba(text_vec)[0]
        
        # Format response
        return PredictionResponse(
            text=request.text,
            sentiment=metadata['classes'][prediction],
            confidence=float(probabilities[prediction]),
            probabilities={
                'negative': float(probabilities[0]),
                'positive': float(probabilities[1])
            },
            timestamp=datetime.now().isoformat()
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Batch prediction endpoint
@app.post("/predict_batch", response_model=BatchResponse)
async def predict_batch(request: BatchRequest):
    """Predict sentiment for multiple texts"""
    try:
        predictions = []
        
        for text in request.texts:
            text_vec = vectorizer.transform([text])
            prediction = model.predict(text_vec)[0]
            probabilities = model.predict_proba(text_vec)[0]
            
            predictions.append(
                PredictionResponse(
                    text=text,
                    sentiment=metadata['classes'][prediction],
                    confidence=float(probabilities[prediction]),
                    probabilities={
                        'negative': float(probabilities[0]),
                        'positive': float(probabilities[1])
                    },
                    timestamp=datetime.now().isoformat()
                )
            )
        
        return BatchResponse(
            predictions=predictions,
            count=len(predictions)
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Model info endpoint
@app.get("/model_info")
async def model_info():
    """Get model metadata and statistics"""
    return metadata

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# Save FastAPI app to file
with open('main.py', 'w') as f:
    f.write(fastapi_app_code)

print("📝 FastAPI App Created!\n")
print("="*70)
print("\n✅ Saved to: main.py")
print("\n🚀 To run the FastAPI service:")
print("   uvicorn main:app --reload")
print("\n📡 API will run on: http://localhost:8000")
print("📚 Auto-generated docs: http://localhost:8000/docs")
print("📖 Alternative docs: http://localhost:8000/redoc")
print("\n🌟 FastAPI Features:")
print("   ✅ Interactive Swagger UI at /docs")
print("   ✅ Automatic request validation")
print("   ✅ Type hints and error handling")
print("   ✅ Production-ready performance")
print("\n" + "="*70)

## 🐳 Step 4: Docker Containerization

**Docker = Package your app + dependencies into a container**

### 🎯 Why Docker?

✅ **"It works on my machine"** → "It works everywhere"  
✅ **Reproducible**: Same environment every time  
✅ **Portable**: Run anywhere (local, cloud, edge)  
✅ **Isolated**: No dependency conflicts  
✅ **Scalable**: Easy to replicate and scale  

### 🏗️ Docker Concepts:

**1. Image**: Blueprint for container  
**2. Container**: Running instance of image  
**3. Dockerfile**: Instructions to build image  
**4. Registry**: Store and share images (Docker Hub)  

### 📦 ML Deployment with Docker:

```
Dockerfile
    ↓
Docker Build
    ↓
Docker Image (app + model + dependencies)
    ↓
Docker Run
    ↓
Container (isolated, reproducible environment)
```

### 🌟 Production Benefits:

- **Development**: Same as production
- **Testing**: Isolated test environment
- **Deployment**: Push to cloud (AWS, GCP, Azure)
- **Scaling**: Spin up multiple containers
- **Updates**: Replace containers, zero downtime

Let's dockerize our FastAPI app!

In [ ]:
# Create Dockerfile for FastAPI app

dockerfile_content = '''
# Use official Python runtime as base image
FROM python:3.9-slim

# Set working directory in container
WORKDIR /app

# Copy requirements file
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY main.py .
COPY models/ ./models/

# Expose port
EXPOSE 8000

# Health check
HEALTHCHECK --interval=30s --timeout=3s --start-period=5s --retries=3 \\
    CMD curl -f http://localhost:8000/ || exit 1

# Run FastAPI with uvicorn
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
'''

with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content.strip())

print("📝 Dockerfile Created!\n")
print("="*70)

# Create requirements.txt
requirements = '''
fastapi==0.104.1
uvicorn[standard]==0.24.0
pydantic==2.5.0
scikit-learn==1.3.2
joblib==1.3.2
numpy==1.24.3
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements.strip())

print("\n✅ Files created:")
print("   📄 Dockerfile")
print("   📄 requirements.txt")

# Create .dockerignore
dockerignore = '''
__pycache__
*.pyc
*.pyo
*.pyd
.Python
env/
venv/
.git
.gitignore
*.ipynb
.ipynb_checkpoints
*.md
'''

with open('.dockerignore', 'w') as f:
    f.write(dockerignore.strip())

print("   📄 .dockerignore")

# Create docker-compose.yml for easy deployment
docker_compose = '''
version: '3.8'

services:
  sentiment-api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - PYTHONUNBUFFERED=1
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/"]
      interval: 30s
      timeout: 3s
      retries: 3
'''

with open('docker-compose.yml', 'w') as f:
    f.write(docker_compose.strip())

print("   📄 docker-compose.yml")

print("\n🐳 Docker Setup Complete!")
print("\n📋 Docker Commands:")
print("\n1️⃣ Build Docker image:")
print("   docker build -t sentiment-api .")
print("\n2️⃣ Run container:")
print("   docker run -p 8000:8000 sentiment-api")
print("\n3️⃣ Or use docker-compose:")
print("   docker-compose up")
print("\n4️⃣ Stop container:")
print("   docker-compose down")
print("\n5️⃣ Push to Docker Hub:")
print("   docker tag sentiment-api username/sentiment-api:v1")
print("   docker push username/sentiment-api:v1")
print("\n" + "="*70)
print("\n💡 Now your API can run ANYWHERE that supports Docker!")

## 🎯 Real AI Example: Complete Deployment Pipeline

**Let's deploy a real sentiment analysis model using transformers!**

This example shows a production-ready deployment using:
- Pre-trained BERT model from HuggingFace
- FastAPI for the API
- Docker for containerization
- Proper error handling and logging

In [ ]:
# Production-ready FastAPI app with HuggingFace model

production_app = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from transformers import pipeline
from typing import List, Optional
import logging
from datetime import datetime
import time

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize FastAPI
app = FastAPI(
    title="Production Sentiment Analysis API",
    description="Real-time sentiment analysis using BERT",
    version="2.0.0",
    docs_url="/docs",
    redoc_url="/redoc"
)

# Load model at startup (cached)
logger.info("Loading transformer model...")
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=-1  # Use CPU (-1) or GPU (0)
)
logger.info("Model loaded successfully!")

# Request/Response models
class SentimentRequest(BaseModel):
    text: str = Field(..., min_length=1, max_length=512)
    
class SentimentResponse(BaseModel):
    text: str
    label: str
    score: float
    processing_time: float
    timestamp: str

class BatchRequest(BaseModel):
    texts: List[str] = Field(..., min_items=1, max_items=100)

class HealthResponse(BaseModel):
    status: str
    model: str
    version: str
    uptime: str

# Startup time
START_TIME = time.time()

@app.get("/", response_model=HealthResponse)
async def health_check():
    """Health check endpoint"""
    uptime = time.time() - START_TIME
    return HealthResponse(
        status="healthy",
        model="distilbert-base-uncased-finetuned-sst-2-english",
        version="2.0.0",
        uptime=f"{uptime:.2f}s"
    )

@app.post("/analyze", response_model=SentimentResponse)
async def analyze_sentiment(request: SentimentRequest):
    """Analyze sentiment of text"""
    try:
        start_time = time.time()
        
        # Run inference
        result = sentiment_pipeline(request.text)[0]
        
        processing_time = time.time() - start_time
        
        logger.info(f"Processed: {request.text[:50]}... ({processing_time:.3f}s)")
        
        return SentimentResponse(
            text=request.text,
            label=result['label'],
            score=result['score'],
            processing_time=processing_time,
            timestamp=datetime.now().isoformat()
        )
    
    except Exception as e:
        logger.error(f"Error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/analyze_batch")
async def analyze_batch(request: BatchRequest):
    """Analyze multiple texts"""
    try:
        start_time = time.time()
        
        # Batch inference
        results = sentiment_pipeline(request.texts)
        
        processing_time = time.time() - start_time
        
        responses = [
            {
                "text": text,
                "label": result['label'],
                "score": result['score']
            }
            for text, result in zip(request.texts, results)
        ]
        
        return {
            "results": responses,
            "count": len(responses),
            "processing_time": processing_time,
            "avg_time_per_text": processing_time / len(request.texts)
        }
    
    except Exception as e:
        logger.error(f"Batch error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/metrics")
async def get_metrics():
    """Get API metrics"""
    return {
        "uptime_seconds": time.time() - START_TIME,
        "model_loaded": True,
        "status": "operational"
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")
'''

with open('production_app.py', 'w') as f:
    f.write(production_app.strip())

print("🏭 Production-Ready API Created!\n")
print("="*70)
print("\n✅ Saved to: production_app.py")
print("\n🌟 Features:")
print("   ✅ Uses pre-trained BERT model")
print("   ✅ Comprehensive logging")
print("   ✅ Error handling")
print("   ✅ Performance metrics")
print("   ✅ Batch processing")
print("   ✅ Health checks")
print("\n🚀 Run with: uvicorn production_app:app --reload")
print("\n" + "="*70)

## 🎯 Interactive Exercises

**Practice your deployment skills!**

### Exercise 1: Deploy Your Own Model

**Task:** Train and deploy a classification model

**Requirements:**
1. Train a model (any sklearn classifier)
2. Save it using joblib
3. Create a FastAPI endpoint
4. Add input validation
5. Test with sample data

**Model ideas:**
- Iris flower classification
- Spam detection
- Price prediction

**Bonus:** Containerize with Docker!

In [ ]:
# YOUR SOLUTION HERE

# TODO: Train your model
# from sklearn.datasets import load_iris
# from sklearn.ensemble import RandomForestClassifier

# TODO: Save your model
# joblib.dump(model, 'my_model.joblib')

# TODO: Create FastAPI app
# (Create a new .py file with FastAPI code)

print("Complete the exercise above!")
print("\nHints:")
print("1. Use a dataset from sklearn.datasets")
print("2. Train any classifier (RandomForest, SVM, etc.)")
print("3. Follow the FastAPI pattern from earlier examples")
print("4. Test your API with curl or requests library")

### Exercise 2: Add Features to API

**Task:** Enhance the sentiment API with additional features

**Add these features:**
1. **Rate limiting**: Limit requests per minute
2. **Caching**: Cache recent predictions
3. **Logging**: Log all predictions to file
4. **Authentication**: Add API key validation
5. **Metrics**: Track request count, avg latency

**Bonus:** Add CORS for frontend integration!

In [ ]:
# YOUR SOLUTION HERE

# Example: Add rate limiting
'''
from slowapi import Limiter, _rate_limit_exceeded_handler
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded

limiter = Limiter(key_func=get_remote_address)
app.state.limiter = limiter
app.add_exception_handler(RateLimitExceeded, _rate_limit_exceeded_handler)

@app.post("/predict")
@limiter.limit("10/minute")
async def predict(request: Request, data: PredictionRequest):
    # Your code here
    pass
'''

print("Complete the exercise above!")
print("\nLibraries to explore:")
print("- slowapi (rate limiting)")
print("- cachetools (caching)")
print("- python-multipart (file uploads)")
print("- fastapi.middleware.cors (CORS)")

## 🎉 Key Takeaways

**Congratulations! You've mastered ML model deployment!**

### 1️⃣ **Model Serialization**
   - ✅ Save models with joblib/pickle
   - ✅ Version control for models
   - ✅ Save metadata for reproducibility
   - **Use when:** Moving models from training to production

### 2️⃣ **Flask APIs**
   - ✅ Simple and lightweight
   - ✅ Great for prototypes
   - ✅ Easy to learn
   - **Use when:** Building MVP or simple APIs

### 3️⃣ **FastAPI (Recommended)**
   - ✅ Production-ready performance
   - ✅ Auto-generated documentation
   - ✅ Type validation with Pydantic
   - ✅ Async support
   - **Use when:** Building production APIs (always!)

### 4️⃣ **Docker Containers**
   - ✅ Reproducible environments
   - ✅ Easy deployment anywhere
   - ✅ Isolation and security
   - **Use when:** Deploying to production (essential!)

---

## 🌟 Real-World Impact

**Skills you can apply immediately:**

### 💼 **Career Skills**
- Deploy ML models as REST APIs
- Build production-ready FastAPI services
- Containerize applications with Docker
- Design scalable ML architectures

### 🏗️ **Deployment Patterns**

**1. Simple Deployment**
```
Train Model → Save → FastAPI → Heroku/Railway
```

**2. Production Deployment**
```
Train → Save → Docker → AWS ECS/K8s → Load Balancer
```

**3. Serverless**
```
Train → Save → AWS Lambda → API Gateway
```

---

## 📊 Best Practices

### ✅ **DO:**
- Use FastAPI for new projects
- Containerize with Docker
- Add health check endpoints
- Log predictions for monitoring
- Version your models
- Validate inputs thoroughly
- Handle errors gracefully
- Test before deploying

### ❌ **DON'T:**
- Deploy without testing
- Ignore error handling
- Skip input validation
- Use Flask for large-scale production
- Hard-code configurations
- Forget about monitoring
- Deploy without versioning

---

## 🚀 Next Steps

**Continue your deployment journey:**

1. **Day 2: MLOps Best Practices**
   - Model versioning with MLflow
   - Experiment tracking
   - Model monitoring
   - A/B testing

2. **Day 3: Cloud Deployment**
   - AWS, GCP, Azure deployment
   - Serverless ML
   - Hugging Face Spaces
   - Streamlit apps

3. **Practice Projects:**
   - Deploy image classifier API
   - Build recommendation system API
   - Create chatbot API
   - Deploy NLP models

---

**💬 Final Thoughts:**

*"A model that isn't deployed creates ZERO value. You now have the skills to take ML models from Jupyter notebooks to production APIs that can serve millions of users. FastAPI + Docker is the modern standard for ML deployment in 2024-2025. Master these tools, and you'll be job-ready for ML engineering roles!"*

**🎉 Day 1 Complete! Tomorrow: MLOps Best Practices! 🚀**

---

**📚 Additional Resources:**
- FastAPI Docs: https://fastapi.tiangolo.com
- Docker Docs: https://docs.docker.com
- HuggingFace Model Hub: https://huggingface.co/models
- Deployment Guide: https://ml-ops.org

**Keep deploying! 🌟**